In [1]:
import re

def clean_text(raw_text):
    # Remove figure/table captions
    text = re.sub(r'(Figure|Table)\s*\d+.*?\n', '', raw_text)
    # Remove references section
    text = re.sub(r'References[\s\S]*', '', text)
    # Remove page numbers
    text = re.sub(r'\n?\s*\d+\s*\n', '\n', text)
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [2]:
cleaned_text = clean_text("book1.pdf")

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

pdf_text = extract_text_from_pdf(cleaned_text)
print(pdf_text[:1000])  # preview


INTRODUCTION 
TO 
LINEAR ALGEBRA 
Third Edition 
GILBERT STRANG 
Massachusetts Institute of Technology 
WELLES LEY·(AMBRI DGE PRESS 
Box 812060 Wellesley MA 02482 
introduction to Linear Algebra, 3rd Edlllon 
Copyrigh~ C>2OO3 by GUbe" Strang 
All righlS ~seryed. No pan of this work may be "'produced or storw or transmitted by any 
moans. including photocopying. without wrinen permin ion from WeJJesley--CllIIIbridge fUss. 
Translation in any language is strictly prohibited 
authorized tBllSlations are amoged. 
Printed in the United States of Amoriea 
ISBN 0·9614088-9-8 
QAIS4.S78 
2003 
5 12'.5 
93-14092 
Other tulS from Wt lles~y-Cambrldge Press 
Wa"elels and Filter Banks, Gilben Strang and Truong Ngu)·cn. 
ISBN 0-%14088·7-1. 
Llnfllr Algebra, Geodny, and GPS, Gilben Strang and Kai Borre. 
ISBN 0-%14088-6-3. 
Introduction 10Applied Mathemali .... Gilbe" Strang. 
ISBN 0-%14088~-4 . 
An Analysis orthe Flnltt Element Method. Gilben Strang and George FiJt. 
ISBN 0·%14088·8-X. 
Calculus. Gi

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len,
)

chunks = splitter.split_text(pdf_text)
print(f"Total chunks: {len(chunks)}")


Total chunks: 1592


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embedding,  # ✅ embedding object, not a function
    collection_name="spectral_analysis_local",
    persist_directory="./chroma_local"
)


In [9]:
#query = "Explain the relationship between spectral analysis and Fourier transform."
query = "Explain the correlation between matrix diagonalization and its eigenvalues and eigenvectors."
results = vectorstore.similarity_search(query, k=3)

context = "\n\n".join([r.page_content for r in results])
print("Context:\n", context[:800])


Context:
 259 
6 
Eigenvalues and Eigenvectors 
274 
6.1 
Introduction to Eigenvalues 
27. 
6.2 
Diagonalizing a Manu 
288 
63 
Applications to Differential Equations 
304 
6A 
Symmetric Matrices 
318 
iii 
6.S 
Positive Definite Matrices 
6.6 
Similar Matrices 
6.7 
Singular Value Decomposition (SVD) 
7 
linear Transformations 
7.1 
The Idea of a Linear Transformation 
7.2 
The Matrix of a Linear Transfonnation 
7.3 
Change of Basis 
7.4 
Diagona.lization and the Pseudoinverse 
8 
Applications 
8.1 
Matrices in Engineering 
8.2 
Graphs and Networks 
8.3 
Markov Matrices and Economic Models 
8.4 
Linear Programming 
8.S 
Fourier Series: Linear Algebra for Functions 
8.6 
Computer Graphics 
9 
Numerical linear Algebra 
9.1 
Gaussian Elimination in Practice 
9.2 
Norms and Condition Numbers 
9.3

259 


In [ ]:
# in the BASH: ollama pull llama3 (pull the model)

In [10]:
import subprocess

#prompt = f"""
#You are an expert in signal processing.
#Using the context below, answer the question clearly.

#Context:
#{context}

#Question: {query}
#"""

prompt = f"""
You are and expert in matrix diagonalization, eigenvalues and eigenvectors.
Using the context below, answer the question clearly.

Context:
{context}

Question: {query}
"""

# Send prompt to local Llama 3 model
result = subprocess.run(
    ["ollama", "run", "llama3"],
    input=prompt.encode("utf-8"),
    capture_output=True
)

print("\n--- Llama 3 Answer ---\n")
print(result.stdout.decode("utf-8"))



--- Llama 3 Answer ---

In the given context, it seems that you are discussing the diagonalization of a matrix B using singular value decomposition (SVD). The equation (4) implies that when the matrix 8 is diagonalized, one of its eigenvalues goes to zero.

The example provided illustrates the importance of eigenvalues and eigenvectors in matrix diagonalization. It highlights how the off-diagonal entries of the matrix B' set the speed of convergence, indicating that the eigenvalues (or singular values) play a crucial role in this process.

The key takeaways from this explanation are:

1. **Eigenvalues influence diagonalization**: The values on the diagonal of the matrix after diagonalization (B') depend on the original matrix's eigenvalues (or singular values).
2. **Eigenvectors matter too**: When diagonalizing a matrix, it is essential to have independent eigenvectors as a basis. Without this, diagonalization becomes impossible.
3. **Eigenvalue convergence**: The example shows that w

In [6]:
#query = "Explain the relationship between spectral analysis and Fourier transform."
query = "Give me example of a matrix, then find its eigenvalues and eigenvectors. After that construct the matrix diagonalization."
results = vectorstore.similarity_search(query, k=3)

context = "\n\n".join([r.page_content for r in results])
print("Context:\n", context[:800])


Context:
 6 
If the eigenvectors of A are the columns of I. then A is a 
matri~. If the 
eigenvector matrix 5 is triangular. then 5- t is triangular. Prove that A is also 
triangular. 
7 
Describe all matrices 5 that diagonalize this matrix A: 
A:=[~ ~l 
Then describe all matrices that diagonalize A - I. 
8 
Write down the most genel1l1 matm thaI has eigenvectors [11 and [-1. 
Questions 9-14 are about Fibonacd and Glbonaccl numbers. 
9 
For the Fibonacci matrix A := [I A], compute A2 and Al and A4. Then use the 
text and a calculator to find FXj. 
10 
Suppose each number GH2 is the o"erage of the two previous numbers Gt+! 
and Gt. Then G1+2 := HG1+1 + Gt): 
GHl "" iGHI + ! Gt 
GHI =GHt 
" 
(a) 
Find the eigenvalues and eigenvectors of A. 
(b)

6 
If the eigenvectors of A are the columns of I. then A


In [7]:
import subprocess

prompt = f"""
You are and expert in matrix diagonalization, eigenvalues and eigenvectors.
Using the context below, answer the question clearly.

Context:
{context}

Question: {query}
"""

# Send prompt to local Llama 3 model
result = subprocess.run(
    ["ollama", "run", "llama3"],
    input=prompt.encode("utf-8"),
    capture_output=True
)

print("\n--- Llama 3 Answer ---\n")
print(result.stdout.decode("utf-8"))



--- Llama 3 Answer ---

Let's consider the following matrix:

A = [[1, 1],
     [0, 1]]

This is known as the Fibonacci matrix.

(a) Finding Eigenvalues and Eigenvectors:
To find the eigenvalues and eigenvectors of A, we can solve the characteristic equation:

det(A - λI) = 0

where λ is an eigenvalue and I is the identity matrix. Expanding the determinant, we get:

(1-λ) + (1-λ) = 0
2-2λ = 0
λ = 1

So, the only eigenvalue of A is λ = 1.

To find the eigenvectors, we can solve the system:

(A - λI)v = 0

where v is an eigenvector. Since λ = 1, we have:

([1, 1] - [1])v = 0
[0, 1]v = 0

The first equation gives us v1 + v2 = 0, and the second equation gives us v2 = 0. This implies that v1 = -v2.

So, the eigenvectors of A are:

v1 = [-1]
v2 = [1]

(b) Matrix Diagonalization:
To diagonalize A, we need to find a matrix S such that S^(-1)AS is diagonal. Since A has only one eigenvalue λ = 1, we can use the eigenvectors v1 and v2 to construct the matrix S:

S = [[-1, 1],
     [0, 1]]

Now, 

In [8]:
query = "give example of system of nonlinear dynamical system. Find its fixed points, then use linearization to find the solution near the fixed point. For this, you need to find the matrix as the result of the linearization, then find the eigenvalues and eigenvectors of the matrix. After that you can construct the matrix diagonalization. Then you can find the solution of the system around the fixed point."
results = vectorstore.similarity_search(query, k=3)

context = "\n\n".join([r.page_content for r in results])
print("Context:\n", context[:800])

Context:
 changes. In nonlinear t(juatioos. It changes as U changes. We don't ha"e either of 
these difficulties. Equation (2) is "linear with coostanl coefficients". 1'1Iose and ooly 
those an:: the differential equations that we will oonl"en directly 10 linear algebra. "The 
main point will be: Sol"" linell, tOIlJkmt cocfficint equalit.>lIs by txlHllfelflu.u c'"',I;. 
Solution "f dll jdr _ Au 
Our pure ",,]>Onential solution will be ~ times a fi~ed vector ,l;. You may guess that 
A is an eigen"aluc of A. and ;r is the eigenl'ector. Substitute U(I) = e).' ;r inlo the 
t(juation du ldl = All 10 pn.",e ~ou art' righl (the foctor t» will cancel): 
Au = A';" ,I; 
ag=' wnh 
d, 
" 
-
"" Ae ;r 
provided 
A;r = J.;r 
d, 
(l) 
, 
i

changes. In nonlinear t(juatioos. It changes as U changes. We don't ha"e ei


In [9]:
import subprocess

prompt = f"""
You are and expert in nonlinear dynamical system, matrix diagonalization, eigenvalues and eigenvectors.
Using the context below, answer the question clearly.

Context:
{context}

Question: {query}
"""

# Send prompt to local Llama 3 model
result = subprocess.run(
    ["ollama", "run", "llama3"],
    input=prompt.encode("utf-8"),
    capture_output=True
)

print("\n--- Llama 3 Answer ---\n")
print(result.stdout.decode("utf-8"))



--- Llama 3 Answer ---

A great question!

Let's consider a simple example of a nonlinear dynamical system:

Suppose we have a population growth model with two species, rabbits (R) and wolves (W). The population dynamics are governed by the following system of differential equations:

dR/dt = rR(1 - R/K) - aRW
dW/dt = wW(-b + cRW)

where:

* R is the rabbit population
* W is the wolf population
* r, K, a, b, and c are parameters representing the growth rate of rabbits, carrying capacity, predation rate, birth rate of wolves, and interaction coefficient between rabbits and wolves, respectively.

This system exhibits nonlinear behavior because the interactions between rabbits and wolves are nonlinear. For example, when there are many rabbits, the predation rate increases quadratically with the rabbit population, which is a nonlinear effect.

Now, let's find the fixed points (equilibrium populations) of this system. A fixed point occurs when the population growth rates become zero:

dR/d

# 🧮 Example: Nonlinear Dynamical System and Linearization Near a Fixed Point

We will consider a **nonlinear two-dimensional system**:

\[
\begin{cases}
\dot{x} = x - x^3 - xy, \\[6pt]
\dot{y} = -2y + x^2.
\end{cases}
\]

We will:
1. Find the **fixed points**.
2. Compute the **Jacobian** and **linearize** the system.
3. Find **eigenvalues** and **eigenvectors** of the Jacobian.
4. Construct the **matrix diagonalization**.
5. Use it to find the **approximate solution** near the fixed point.

---

## 1️⃣ Find Fixed Points

At equilibrium, \(\dot{x} = 0\) and \(\dot{y} = 0\):

\[
\begin{cases}
x - x^3 - xy = 0, \\[6pt]
-2y + x^2 = 0.
\end{cases}
\]

From the second equation:
\[
y = \dfrac{x^2}{2}.
\]

Substitute into the first equation:
\[
x - x^3 - x\frac{x^2}{2} = x\left(1 - \frac{3}{2}x^2\right) = 0.
\]

Hence:
\[
x = 0 \quad \text{or} \quad 1 - \frac{3}{2}x^2 = 0 \Rightarrow x = \pm \sqrt{\frac{2}{3}}.
\]

The fixed points are:
\[
(0, 0), \quad \left(\sqrt{\tfrac{2}{3}}, \tfrac{1}{3}\right), \quad \left(-\sqrt{\tfrac{2}{3}}, \tfrac{1}{3}\right).
\]

We will analyze the **positive fixed point**:
\[
(x^*, y^*) = \left(\sqrt{\tfrac{2}{3}}, \tfrac{1}{3}\right).
\]

---

## 2️⃣ Linearization — Compute the Jacobian

The Jacobian matrix \( J(x, y) \) is:

\[
J(x, y) = 
\begin{pmatrix}
\dfrac{\partial f_1}{\partial x} & \dfrac{\partial f_1}{\partial y} \\[6pt]
\dfrac{\partial f_2}{\partial x} & \dfrac{\partial f_2}{\partial y}
\end{pmatrix}
=
\begin{pmatrix}
1 - 3x^2 - y & -x \\[6pt]
2x & -2
\end{pmatrix}.
\]

Evaluate at the fixed point \((x^*, y^*) = (\sqrt{2/3}, 1/3)\):

\[
J^* = 
\begin{pmatrix}
1 - 3\left(\frac{2}{3}\right) - \frac{1}{3} & -\sqrt{\frac{2}{3}} \\[6pt]
2\sqrt{\frac{2}{3}} & -2
\end{pmatrix}
=
\begin{pmatrix}
-\frac{4}{3} & -\frac{\sqrt{6}}{3} \\[6pt]
\frac{2\sqrt{6}}{3} & -2
\end{pmatrix}.
\]

Numerically:
\[
J^* \approx 
\begin{pmatrix}
-1.3333 & -0.8165 \\[4pt]
1.6330 & -2
\end{pmatrix}.
\]

---

## 3️⃣ Eigenvalues and Eigenvectors

Solve \( \det(J^* - \lambda I) = 0 \):

\[
\lambda_{1,2} = -\frac{5}{3} \pm i\frac{\sqrt{11}}{3}.
\]

Thus:
\[
\alpha = -\frac{5}{3}, \quad \beta = \frac{\sqrt{11}}{3}.
\]

Since the real part \( \alpha < 0 \), the fixed point is a **stable spiral** (decaying oscillation).

An eigenvector for \( \lambda_1 = \alpha + i\beta \) is approximately:

\[
v = 
\begin{pmatrix}
0.2041 + 0.6770i \\[4pt]
1
\end{pmatrix}.
\]

---

## 4️⃣ Diagonalization of the Linearized Matrix

The complex diagonalization form is:

\[
J^* = P D P^{-1},
\]
where
\[
P = [\,v \ \ \overline{v}\,], \quad D = 
\begin{pmatrix}
\lambda_1 & 0 \\[4pt]
0 & \lambda_2
\end{pmatrix}.
\]

The general (complex) solution of the linearized system
\[
\dot{z} = J^* z
\]
is:
\[
z(t) = P e^{D t} P^{-1} z(0),
\]
where \( e^{D t} = \operatorname{diag}(e^{\lambda_1 t}, e^{\lambda_2 t}) \).

---

## 5️⃣ Real Solution Near the Fixed Point

Let \( v = a + i b \), where

\[
a = \Re(v) = 
\begin{pmatrix}
0.2041 \\[4pt] 1
\end{pmatrix}, \quad
b = \Im(v) = 
\begin{pmatrix}
0.6770 \\[4pt] 0
\end{pmatrix}.
\]

Then the **real solution** near the fixed point is:

\[
z(t) = e^{\alpha t} 
\left[
C_1(a\cos(\beta t) - b\sin(\beta t)) +
C_2(a\sin(\beta t) + b\cos(\beta t))
\right],
\]
where \( C_1, C_2 \in \mathbb{R} \) are constants determined by the initial deviation \( z(0) \).

Finally:
\[
(x(t), y(t)) \approx (x^*, y^*) + z(t).
\]

---

## 🧩 Interpretation

- The eigenvalues have **negative real parts**, so trajectories **spiral toward** the fixed point.
- The oscillation frequency is \(\beta = \frac{\sqrt{11}}{3} \approx 1.1055\).
- The exponential decay rate is \(\alpha = -\frac{5}{3}\).

✅ Therefore, the fixed point \((x^*, y^*) = \left(\sqrt{\tfrac{2}{3}}, \tfrac{1}{3}\right)\) is a **stable spiral**.

---

*Tip:*  
In Jupyter, you can render this beautifully by setting the cell type to **Markdown** and pressing `Shift + Enter`.  
Make sure you enable **LaTeX rendering** in Jupyter for the equations.
